# Basic imports for Mask R-CNN

In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

from skimage import transform
import skimage.io


%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

DATA_DIR = os.path.join(ROOT_DIR, "../data")

MASK_DIR  = os.path.join(DATA_DIR,'segmentation')

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(DATA_DIR, "mask_rcnn_coco.h5")

#Make GPUs visible
!export HIP_VISIBLE_DEVICES=1,2,3

#Set which GPU devices' memory should be accessible to running GPUs
os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3"

#Make only 1 GPU visible
# !export HIP_VISIBLE_DEVICES=2,3

#Set which GPU devices' memory should be accessible to running GPUs
# os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

# Download COCO trained weights from Releases if needed
# if not os.path.exists(COCO_MODEL_PATH):
#     utils.download_trained_weights(COCO_MODEL_PATH)

def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

Using TensorFlow backend.


# Pull in annotations from database (from Simplot)

In [ ]:
import pyodbc
import os
import pandas as pd

DATA_DIR = os.path.join(ROOT_DIR, "../data")

COCO_MODEL_PATH = os.path.join(DATA_DIR, "mask_rcnn_coco.h5")

ANNOTATION_COLS = ['XMax','XMin','YMin','YMax','LabelDescription']

passwd = os.environ['dgs_sandbox_pwd']

conn = pyodbc.connect('DSN=BIdevDatabase;'
                      'Database=Sandbox;'
                      'UID=DGSuser;'
                      'PWD=' + passwd)


class_descriptions = pd.read_csv(os.path.join(DATA_DIR, "seg_class_descriptions.csv"))

#add 1 since Background class is automatically added at index 0
class_descriptions['LabelID'] = class_descriptions.index + 1

#This now holds the annotation information.
# bboxes = pd.read_sql("SELECT ImageID, XMax, XMin, YMin, YMax, LabelName FROM [Sandbox].[kaggle].[Combined_Set_Detection_BBox]", conn)

seg_anns = pd.read_sql("SELECT MaskPath, ImageID, LabelName, BoxID, BoxXMin, BoxXMax, BoxYMin, BoxYMax, PredictedIoU, Clicks, SourceDataset from [kaggle].[Combined_Annotations_Object_Segmentation]", conn)

annotations = pd.merge(seg_anns,class_descriptions, on='LabelName',how='inner')

# This now holds the list of images
image_paths = pd.read_sql("SELECT ImageID, RelativePath, Height, Width, Mode from [kaggle].[Image_Path]", conn)

# Inner join on the two dataframes, so we now have images combined with associated annotations
annotated_image_paths = pd.merge(image_paths,annotations, on='ImageID',how='inner')

display(annotated_image_paths.head())

# Configurations

In [ ]:
class KaggleConfig(Config):
    
    NAME = "kaggle"

    #TODO experiment with this
    GPU_COUNT = 3
    IMAGES_PER_GPU = 4  # TODO how many can we use, authors had 2 for 12 GB, we have 32 GB so ...
    
    # Number of classes (including background)
    NUM_CLASSES = 1 + len(class_descriptions)  # + 1 for background

    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.85
    
config = KaggleConfig()
config.display()

# Subclass utils.Dataset for entire experiment

In [4]:
class SegKaggleImageDataset(utils.Dataset):
         
    def load_kaggle_images(self, dataset_dir, grouped_by_images):
        """Load a subset of the image dataset.
        dataset_dir: The root directory of the image dataset.
        classes: Dataframe : If provided, only loads images that have the given classes.
        """

        # Add classes, BG is automatically added at index 0
        for _,row in class_descriptions.iterrows():
            self.add_class("openImages", row['LabelID'], row['LabelDescription'])
        
        # Add images
        for i,df in grouped_by_images:    
            row = df.iloc[0]
            
            self.add_image(
                "openImages", image_id=i,
                path=os.path.join(dataset_dir, row['RelativePath']),
                width=row["Width"],
                height=row["Height"],
                annotations=df)
    

    def load_mask(self, image_id):
        """Generate instance masks for an image.  Based on the nucleus sample
        
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        
        # Create rectangular bounding box since we are doing object detection, not segmentation
        # desired dimension is [height, width, instance_count]
        img = self.image_info[image_id]
        
        masks = []
        
        for i,(_,p) in enumerate(img["annotations"].iterrows()):   
            mpath = os.path.join(MASK_DIR, p['MaskPath'])
            raw_mask = skimage.io.imread(mpath)
            # mask is often not the same size as the image, resize the mask so that it is
            masks.append(transform.resize(raw_mask,(img['height'],img['width'])))

        mask = np.stack(masks, axis=-1)
        
        return mask,np.array(img['annotations']['LabelID'].values, dtype=np.int32)

    def image_reference(self, image_id):
        return self.image_info[image_id]['path']
    

# Load dataset and display samples

## Instantiate the Full Kaggle Image Dataset classes

In [5]:
def load_dataset(source_str):
    seg_paths = annotated_image_paths[annotated_image_paths['SourceDataset'] == source_str]

    seg_grouped = seg_paths.groupby('ImageID')
    
    dataset_seg = SegKaggleImageDataset()
    dataset_seg.load_kaggle_images(DATA_DIR, seg_grouped)
    dataset_seg.prepare()
    
    return dataset_seg


dataset_train = load_dataset('train')
dataset_val = load_dataset('validation')

### Sanity check that masks look alright

In [ ]:
# Load and display random samples ... sanity check for data load
def display_mask_samples(dataset):
    image_ids = np.random.choice(dataset.image_ids, 5)

    for image_id in image_ids:
        image = dataset.load_image(image_id)
        mask, class_ids = dataset.load_mask(image_id)
        visualize.display_top_masks(image, mask, class_ids, dataset.class_names)
        
display_mask_samples(dataset_val)

# Training

## Create model and load prior weights

In [12]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [13]:
# Which weights to start with?
init_with = "last"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

Re-starting from epoch 600


## Train

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

In [ ]:
# !pip3 install imgaug

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. Passing layers="all" trains all layers. You can also pass a regular expression to select
# which layers to train by name pattern.

import imgaug

# Image Augmentation ... pulled from coco example
# Right/Left flip 50% of the time
augmentation = imgaug.augmenters.Fliplr(0.5)

train_stage = 2
n_epochs = 700 #starts from last round's end number

if train_stage == 1:
    model.train(dataset_train, dataset_val, 
                learning_rate= config.LEARNING_RATE, 
                epochs=n_epochs, 
                layers='heads',
                augmentation=augmentation)
elif train_stage == 2:
    model.train(dataset_train, dataset_val, 
                learning_rate= config.LEARNING_RATE/10, 
                epochs=n_epochs, 
                layers='4+',
                augmentation=augmentation)
elif train_stage == 3:
    model.train(dataset_train, dataset_val, 
                learning_rate= config.LEARNING_RATE / 10, 
                epochs=n_epochs, 
                layers='all',
                augmentation=augmentation)


Starting at epoch 600. LR=0.0001

Checkpoint Path: /home/nbuser/repo/logs/kaggle20190913T1140/mask_rcnn_kaggle_{epoch:04d}.h5
Selecting layers to train
res4a_branch2a         (Conv2D)
bn4a_branch2a          (BatchNorm)
res4a_branch2b         (Conv2D)
bn4a_branch2b          (BatchNorm)
res4a_branch2c         (Conv2D)
res4a_branch1          (Conv2D)
bn4a_branch2c          (BatchNorm)
bn4a_branch1           (BatchNorm)
res4b_branch2a         (Conv2D)
bn4b_branch2a          (BatchNorm)
res4b_branch2b         (Conv2D)
bn4b_branch2b          (BatchNorm)
res4b_branch2c         (Conv2D)
bn4b_branch2c          (BatchNorm)
res4c_branch2a         (Conv2D)
bn4c_branch2a          (BatchNorm)
res4c_branch2b         (Conv2D)
bn4c_branch2b          (BatchNorm)
res4c_branch2c         (Conv2D)
bn4c_branch2c          (BatchNorm)
res4d_branch2a         (Conv2D)
bn4d_branch2a          (BatchNorm)
res4d_branch2b         (Conv2D)
bn4d_branch2b          (BatchNorm)
res4d_branch2c         (Conv2D)
bn4d_branch

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple worker

Epoch 601/700
1000/1000 [==============================] - 845s 845ms/step - loss: 1.0375 - rpn_class_loss: 0.0345 - rpn_bbox_loss: 0.3149 - mrcnn_class_loss: 0.2452 - mrcnn_bbox_loss: 0.1849 - mrcnn_mask_loss: 0.2581 - val_loss: 1.6367 - val_rpn_class_loss: 0.0454 - val_rpn_bbox_loss: 0.7823 - val_mrcnn_class_loss: 0.3267 - val_mrcnn_bbox_loss: 0.2476 - val_mrcnn_mask_loss: 0.2347
Epoch 602/700
1000/1000 [==============================] - 706s 706ms/step - loss: 0.9003 - rpn_class_loss: 0.0270 - rpn_bbox_loss: 0.2364 - mrcnn_class_loss: 0.2432 - mrcnn_bbox_loss: 0.1557 - mrcnn_mask_loss: 0.2381 - val_loss: 1.8674 - val_rpn_class_loss: 0.0519 - val_rpn_bbox_loss: 0.9201 - val_mrcnn_class_loss: 0.3553 - val_mrcnn_bbox_loss: 0.2549 - val_mrcnn_mask_loss: 0.2851
Epoch 603/700
1000/1000 [==============================] - 698s 698ms/step - loss: 0.8485 - rpn_class_loss: 0.0285 - rpn_bbox_loss: 0.2400 - mrcnn_class_loss: 0.1956 - mrcnn_bbox_loss: 0.1507 - mrcnn_mask_loss: 0.2337 - val_loss: 

In [ ]:
# epochs [0,500] were stage 1 : 3 gpus, 4 images / gpu, learning rate 0.001
# epochs [501,600] were stage 1 : 4 gpus, 4 images /gpu, learning rate 0.0001
# epochs [601,700] were stage 2 : 3 gpus, 4 images / gpu, learning rate 0.0001


# Detection

In [ ]:
class InferenceConfig(Config):
    NAME = 'KaggleInf'
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + len(class_descriptions)
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, "logs/kaggle20190911T1433/mask_rcnn_kaggle_0200.h5")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

## Sanity Check on validation set

In [ ]:
#test on a few random images
image_ids = np.random.choice(dataset_val.image_ids, 5)

images = []

for iid in image_ids:
    original_image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset_val, inference_config, iid, use_mini_mask=False)

    visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_val.class_names, figsize=(8, 8))
    
    images.append(original_image)


In [ ]:
results = [model.detect([img], verbose=0) for img in images]

for i in range(len(results)):
    r = results[i][0]
    visualize.display_instances(images[i], r['rois'], r['masks'], r['class_ids'], dataset_val.class_names, r['scores'])

## On Test set

In [ ]:
from PIL import ImageFile
import os
ImageFile.LOAD_TRUNCATED_IMAGES = True

#TODO image_info should be 2d array, with each row of form id, width, height
def append_to_file(filename, results, image_info):
    
    all_preds = []
    
    for j, r in enumerate(results):
        preds = []
        ids = r['class_ids']
        boxes = r['rois']
        scores = r['scores']

        img_id = image_info[j][0]
        height = image_info[j][2]
        width = image_info[j][1]
        
        preds = ''

        for i in range(len(r['class_ids'])):
            xmin = max(boxes[i][1] / width , 0.0)
            ymin = max(boxes[i][0] / height, 0.0)
            xmax = min(boxes[i][3] / width, 1.0)
            ymax = min(boxes[i][2] / height, 1.0)
            
            preds += " " + " ".join(map(str,[omni_class_set.iloc[ids[i]]['LabelName'], scores[i], xmin,ymin,xmax,ymax]))

        all_preds.append(img_id + "," + preds)
        
    with open(filename, 'a') as f: 
        f.write('\n'.join(all_preds))
        f.write('\n')
    
    

testdir = os.path.join(DATA_DIR, "test")
batch_size = 500


def write_sub_file(filename, start_index=0):
    results = []
    image_info = []


    for subdir, dirs, files in os.walk(testdir):
        for cnt,file in enumerate(files):

            #use this if the process broke down at some point and you need to restart midway through ... total hack
            if cnt < start_index:
                continue

            img = skimage.io.imread(os.path.join(subdir, file))

            #filename, width, height
            image_info.append([file[:-4],img.shape[1],img.shape[0]])

            results += model.detect([image], verbose=0)

            if (cnt%batch_size == (batch_size-1)):
                print("writing to file ... ")
                append_to_file(filename, results,image_info)
                results = []
                image_info = []
                print(cnt," completed") #100,000 images in the test set
                
    print("writing final records to file ... ")
    append_to_file(filename, results,image_info)
    print(cnt," completed")
    
    return cnt

### Sanity Check

In [ ]:
dataset = dataset_val

image_ids = np.random.choice(dataset.image_ids, 5)

ver_images = []

for iid in image_ids:
    original_image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset, inference_config, iid, use_mini_mask=False)
    
    visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset.class_names, figsize=(8, 8))
    
    ver_images.append(original_image)

results = [model.detect([img], verbose=0) for img in ver_images]

for i in range(len(results)):
    r = results[i][0]
    visualize.display_instances(ver_images[i], r['rois'], r['masks'], r['class_ids'], dataset.class_names, r['scores'])

# Evaluation

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 500)

#each input is a tuple of form : image, image_meta, gt_class_id, gt_bbox, gt_mask
inputs = [modellib.load_image_gt(dataset_val, inference_config, iid, use_mini_mask=False) for iid in image_ids]

APs = []

n = inference_config.BATCH_SIZE

for i in range(0,len(image_ids),n): 

    curr_inputs = inputs[i:i+n]
    
    results = model.detect([inp[0] for inp in curr_inputs], verbose=0)
    
    for j in range(len(results)):
        r = results[j]
        # Compute AP
        AP, precisions, recalls, overlaps =\
            utils.compute_ap(curr_inputs[j][3], curr_inputs[j][2], curr_inputs[j][4],
                             r["rois"], r["class_ids"], r["scores"], r['masks'])
        APs.append(AP)
    
print("mAP: ", np.mean(APs))

# Generate submission file

## Encode instance segmentation map (from kaggle competition website)

In [ ]:
# container does not include this library by default, will need to run this once
# !pip install pycocotools

In [ ]:
import base64
from pycocotools import _mask as coco_mask
import typing as t
import zlib


def encode_binary_mask(mask: np.ndarray):

     # check input mask --
    if mask.dtype != np.bool:
        raise ValueError(
           "encode_binary_mask expects a binary mask, received dtype == %s" %
           mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
           "encode_binary_mask expects a 2d mask, received shape == %s" %
           mask.shape)

     # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str

## Define batch-write functions

In [ ]:
from PIL import ImageFile
import os
ImageFile.LOAD_TRUNCATED_IMAGES = True

#TODO image_info should be 2d array, with each row of form id, width, height
def append_to_file(filename, results, image_info):
    
    all_preds = []
    
    for j, r in enumerate(results):
        preds = []
        ids = r['class_ids']
        masks = r['masks']
        scores = r['scores']

        img_id = image_info[j][0]
        height = image_info[j][2]
        width = image_info[j][1]
        
        preds =''

        for i in range(len(r['class_ids'])):
            # masks are stored as a 3d array, <height,width,# examples>, so we need to index into it in a special way
            enc_mask = encode_binary_mask(masks[:,:,i])
            #subtract 1 to compensate for the background
            class_name = class_descriptions.iloc[ids[i]-1]['LabelName']
            preds += " " + " ".join(map(str,[class_name, scores[i], enc_mask.decode()]))

        img_lvl_fields = ','.join(map(str,[img_id,width,height]))
        all_preds.append(img_lvl_fields + "," + preds)
        
    with open(filename, 'a') as f: 
        f.write('\n'.join(all_preds))
        f.write('\n')
        

testdir = os.path.join(DATA_DIR, "test")

def write_sub_file(filename, batch_size=500, start_index=0):
    results = []
    image_info = []


    for subdir, dirs, files in os.walk(testdir):
        for cnt,file in enumerate(files):

            #use this if the process broke down at some point and you need to restart midway through ... total hack
            if cnt < start_index:
                continue

            img = skimage.io.imread(os.path.join(subdir, file))

            #filename, width, height
            image_info.append([file[:-4],img.shape[1],img.shape[0]])

            results += model.detect([img], verbose=0)

            if (cnt%batch_size == (batch_size-1)):
                print("writing to file ... ")
                append_to_file(filename, results,image_info)
                results = []
                image_info = []
                print(cnt," completed") #100,000 images in the test set
                
    print("writing final records to file ... ")
    append_to_file(filename, results,image_info)
    print(cnt," completed")
    
    return cnt

## Execute batch writing

In [ ]:
filename = os.path.join(ROOT_DIR, 'seg_submission_9_16.csv')

### Generate the entire file

In [ ]:
with open(filename, 'w+') as f:
    f.write('ImageId,ImageWidth,ImageHeight,PredictionString\n')
    
write_sub_file(filename, start_index=0)

### Resume writing the file

In [ ]:
# use X + 1 , where X is from the 'X completed' statement above
write_sub_file(filename, start_index=6500)

### Manipulate submission file

In [ ]:
# df = pd.read_csv(submission_file)

# nu = df['ImageId'].nunique()  # Should be 99999

# if len(df) > nu
#     df.drop_duplicates('ImageId', inplace = True)

# df.to_csv('submission_9_11_0.csv',index=False)

# Debugging cells

In [ ]:
## Look up the details for an image and its associated mask to ensure that sizes are correctly mapped

# mpath = dataset.image_reference(548819)
# mpath = dataset.image_reference(263657)
# skimage.io.imread(mpath).shape

def get_details(img_id):
    f_path = dataset.image_reference(img_id)
    f_shape = skimage.io.imread(f_path).shape
    f_ratio = f_shape[0]/f_shape[1]
    q = f_path.split('/')[-2:]
    rel_m_path = annotated_image_paths[annotated_image_paths['RelativePath'] == '/'.join(q)]['MaskPath'].iloc[0]
    m_path = os.path.join(DATA_DIR,'segmentation',rel_m_path)
    m_shape = skimage.io.imread(m_path).shape
    m_ratio = m_shape[0]/m_shape[1]
    
    return [img_id, f_path,f_shape,f_ratio,m_path,m_shape,m_ratio]

x = [get_details(iid) for iid in image_ids]

pd.DataFrame(x,columns=['iid','f_path','f_shape','f_ratio','m_path','m_shape','m_ratio'])